Dieses Script ist für die Segmentierung von Punktwolken ohne Ground Truth Informationen.

In [1]:
from pyntcloud import PyntCloud
import os
import numpy as np
import random
from random import randint, uniform
import pandas as pd
import tensorflow as tf
import math
import time
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
from collections import defaultdict

from sklearn.metrics import calinski_harabasz_score # the higher the better

import open3d as o3d
import scipy.io as io
import scipy.ndimage as nd


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ User Settings ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Set size of Voxels. The higher the better but can also be very slow with big Pointclouds
voxel_size = 256

# Restore the trained model
if (voxel_size == 64):
    model = tf.keras.models.load_model('savedmodel/scannet_model_64.h5')
    print("CNN Model with Voxelsize of 64 has been loaded.")

if (voxel_size == 128):
    model = tf.keras.models.load_model('savedmodel/scannet_model_128.h5')
    print("CNN Model with Voxelsize of 128 has been loaded.")
    
if (voxel_size == 192):
    model = tf.keras.models.load_model('savedmodel/scannet_model_192.h5')
    print("CNN Model with Voxelsize of 192 has been loaded.")

if (voxel_size == 256):
    model = tf.keras.models.load_model('savedmodel/scannet_model_256.h5')
    print("CNN Model with Voxelsize of 256 has been loaded.")
          
# Clustering number of candidates 0-7
# 0 ^= 1x Gaussian Mixture Candidate and 1x Bayesian Mixture Candidate
# The more points the pointcloud has the lower the nb of candidates should be
n_candidates = 3


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Functions ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# Colorizes the different Clusters randomly
def colorize_pc(number_clusterpoints):

    rgb = np.zeros((number_clusterpoints, 3), dtype=int)

    r = randint(0, 254)
    g = randint(0, 254)
    b = randint(0, 254)

    rgb_temp = np.array((r,g,b))
    rgb = np.full_like(rgb,rgb_temp)

    return rgb

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Adding the Clusterlabel to the Clusters
def add_clusterlabel(number_clusterpoints, clusterlabel):
    
    clusterlabel_array = np.zeros((number_clusterpoints,1), dtype=int)
    clusterlabel_temp = np.array(clusterlabel)
    clusterlabel_array = np.full_like(clusterlabel_array, clusterlabel_temp)
    
    return clusterlabel_array
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Colorize the points and returns a dataframe with rgb and a dataframe with labels
def colorize_to_dataframe(clusters_dict, cnn_prediction):
    
    # Placeholder for the colorized Clusterscene (x,y,z,r,b,g(segment color), cluster_label)
    cluster_scene = np.zeros((0, 7), dtype=float)
    
    clusterlabel = 0
    for clusterlabel in range(cnn_prediction):
        cluster = clusters_dict[clusterlabel]
        number_clusterpoints = cluster.shape[0]
        colorize_rgb = colorize_pc(number_clusterpoints)
        
        # Adding Color to cluster
        cluster = np.hstack((cluster, colorize_rgb))

        #Adding Clusterlabel to Cluster
        cluster_label = add_clusterlabel(number_clusterpoints, clusterlabel)
        cluster = np.hstack((cluster, cluster_label))
        cluster_scene = np.append(cluster_scene, cluster, axis=0)
         
    x = cluster_scene[:, 0]
    y = cluster_scene[:, 1]
    z = cluster_scene[:, 2]
    # Segment rgb Color
    red = cluster_scene[:, 3]
    green = cluster_scene[:, 4]
    blue = cluster_scene[:, 5]
    # Cluster Label
    cluster_label = cluster_scene[:, 6]

    df_cluster_scene_rgb = pd.DataFrame(zip(x, y, z, red, green, blue), columns=["x", "y", "z", "red", "green", "blue"])
    df_cluster_scene_labels = pd.DataFrame(zip(x, y, z, cluster_label), columns=["x", "y", "z","cluster_label"])
    npy_cluster_scene_labels = df_cluster_scene_labels.to_numpy()
        
    return df_cluster_scene_rgb, npy_cluster_scene_labels

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# load data from cvs    
def load_cvs_data(path):
    return np.loadtxt(path,delimiter=';')

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            # relative max occurence of label: arr[j][3][0]/arr[j][1]
            #if arr[j][3][0]/arr[j][1] < arr[j+1][3][0]/arr[j+1][1]:
            if arr[j][3].max() < arr[j+1][3].max():
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def downsample_pointcloud(pointcloud, coeff):
    return pointcloud[::coeff]
    pass

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_down_sample(ind)
    outlier_cloud = cloud.select_down_sample(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def pc_normalize(pointcloud):
    
    l = pointcloud.shape[0]
    centroid = np.mean(pointcloud, axis=0)
    pointcloud = pointcloud - centroid
    m = np.max(np.sqrt(np.sum(pointcloud**2, axis=1)))
    pointcloud = pointcloud / m
    return pointcloud

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def get_assignments(best_clusterscene, clusterscene):

    i = 0
    j = 0
    k = 0

    assignments = defaultdict(dict)
    n_true_points_segment = {}
    seg_max  = 0

    while i < (cnn_prediction): # i = number of segments/objects for best clusterscene

        # Get the first "best" segment
        best_segment = best_clusterscene[np.where(best_clusterscene[:,3] == i)]
        best_segment = np.delete(best_segment, 3, axis=1) # delete unnecessary label information
        
        n_points_best_segment = best_segment.shape[0] # get number of points of segment
       
        while j <= n_candidates: # j = number of Candidates
                   
            while k < (cnn_prediction): # k = number of segments/objects of all clusterings
                
                # get the segment
                segment = clusterscene[j][np.where(clusterscene[j][:,3] == k)]
                segment = np.delete(segment, 3, axis=1)
                
                # count number of matching points 
                result = np.isin(best_segment, segment)
                n_true_points = 0
                for r in result:
                    if(np.all(r)):
                        n_true_points = n_true_points + 1

                # store all true points per segment in array for max calulation
                n_true_points_segment[k] = n_true_points 
                
                # score calculation
                segment_score = (n_true_points_segment[k] / n_points_best_segment) * 100 
                segment_score = round(segment_score)
                
                # Output:  max of matching Points, real number of points, assignment, segment score
                if seg_max == 0:
                    assignments[j][i] = [n_true_points_segment[k]], [best_segment.shape[0]], [i], [k], [segment_score]

                elif n_true_points_segment[k] > seg_max:
                    assignments[j][i] = [n_true_points_segment[k]], [best_segment.shape[0]], [i], [k], [segment_score]

                seg_max = n_true_points_segment[max(n_true_points_segment, key=n_true_points_segment.get)] 
                
                k = k + 1

            n_true_points_segment.clear()
            seg_max  = 0
            k = 0
            j = j + 1
        
        i = i + 1
        j = 0
        k = 0    
    return assignments
    
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def homogenity_check(gm_assignments, bm_assignments):
    i = 0
    j = 0
    gm_score = 0
    bm_score = 0
    
    # First go trough all the Candidates  
    while j <= n_candidates: # j = number of Candidates
        while i < (cnn_prediction): # i = number of Segments / Objects         
            
            # Get the sement score
            gm_segment_score = gm_assignments[j][i][4][0]
            bm_segment_score = bm_assignments[j][i][4][0] 
            
            # += all points in Cluster
            gm_score += gm_segment_score
            bm_score += bm_segment_score
            
            i = i + 1
        
        i = 0
        j = j + 1
    
    homogenity = (((gm_score + bm_score - (cnn_prediction*100))/(((n_candidates + 1)* 2) - 1)) / (cnn_prediction*100))*100
    return round(homogenity,2)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Restore RGB Information
def restore_real_rgb(best_clusterscene, pointcloud):
    
    n_points = best_clusterscene.shape[0]
    final_clusterscene = np.empty((n_points,7))

    bx = best_clusterscene[:,0]
    by = best_clusterscene[:,1]
    bz = best_clusterscene[:,2]
    blabel = best_clusterscene[:,3]

    npy_pointcloud = pointcloud.points.to_numpy()
    
    # Initial Pointcloud after size manipulations
    ix = npy_pointcloud[:,0]
    iy = npy_pointcloud[:,1]
    iz = npy_pointcloud[:,2]
    
    ir = npy_pointcloud[:,3]
    ig = npy_pointcloud[:,4]
    ib = npy_pointcloud[:,5]
    
    i=0
    j=0
    
    # This takes a long time and needs to be simplyfied. 
    
    while i < n_points:
        while j < n_points:
            
            if bx[i] == ix[j] and by[i] == iy[j] and bz[i] == iz[j]:
                temp = [bx[i], by[i], bz[i], ir[j], ig[j], ib[j], blabel[i]]
                final_clusterscene[i] = temp
                i+=1
                j=n_points+1 # Stop the loop when xyz match is found and continue with next i
            else: 
                j+=1
                
                
            """
            intersect = np.intersect1d(npy_best_clusterscene[i], pointcloud.xyz[j], assume_unique=True) # compares the Arrays
            if(intersect.shape[0] == 3):
                temp = np.append(npy_pointcloud[j], blabel[j])
                final_clusterscene[i] = temp
                i += 1   
            """
        j = 0

    return final_clusterscene

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Converts any numpy pointcloud array to a PyntCloud object. 
def numpy_to_pyntcloud(pointcloud, normalized_rgb):
    
    pc_dimension = pointcloud.shape[1]
    
    if(pc_dimension == 0 or pc_dimension == 1 or pc_dimension == 2):
        print("Error. Pointcloud must have at least XYZ-Informations.")
    
    else :
        
        if (pc_dimension == 3):
            print("Creating an X,Y,Z Pyntcloud.")
            x = pointcloud[:, 0]
            y = pointcloud[:, 1]
            z = pointcloud[:, 2]
            
            df_output_pc = pd.DataFrame(zip(x, y, z), columns=['x', 'y', 'z'])
            output_pc = PyntCloud(df_output_pc)
        
        if (pc_dimension == 4):
            print("Creating an X,Y,Z,Label Pyntcloud.")
            x = pointcloud[:, 0]
            y = pointcloud[:, 1]
            z = pointcloud[:, 2]
            label = pointcloud[:, 3]
            
            df_output_pc = pd.DataFrame(zip(x, y, z, label), columns=['x', 'y', 'z','label'])
            output_pc = PyntCloud(df_output_pc)
        
        if (pc_dimension == 5):
            print("Error.Invalid Pointcloud dimension.")
        
        if (pc_dimension == 6):
            print("Creating an X,Y,Z,R,G,B Pyntcloud.")
            x = pointcloud[:, 0]
            y = pointcloud[:, 1]
            z = pointcloud[:, 2]
            
            if (normalized_rgb !=1):
                print("Creating an X,Y,Z,R,G,B Pyntcloud.")
                r = pointcloud[:, 3]
                g = pointcloud[:, 4]
                b = pointcloud[:, 5]
            else:
                print("Creating an X,Y,Z,R,G,B Pyntcloud. With normalized RGB")
                r = pointcloud[:, 3]/255
                g = pointcloud[:, 4]/255
                b = pointcloud[:, 5]/255
                
            
            df_output_pc = pd.DataFrame(zip(x, y, z, r, g, b), columns=['x', 'y', 'z','red', 'green', 'blue'])
            output_pc = PyntCloud(df_output_pc)
        
        if (pc_dimension == 7):
            x = pointcloud[:, 0]
            y = pointcloud[:, 1]
            z = pointcloud[:, 2]
            
            if (normalized_rgb !=1):
                print("Creating an X,Y,Z,R,G,B, Label Pyntcloud.")
                r = pointcloud[:, 3]
                g = pointcloud[:, 4]
                b = pointcloud[:, 5]
            else:
                print("Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.")
                r = pointcloud[:, 3]/255
                g = pointcloud[:, 4]/255
                b = pointcloud[:, 5]/255
                
            label = pointcloud[:, 6]

            df_output_pc = pd.DataFrame(zip(x, y, z, r, g, b, label), columns=['x', 'y', 'z','red', 'green', 'blue','label'])
            output_pc = PyntCloud(df_output_pc)

    return output_pc

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Exports the final Pointcloud to .ply files
def export_to_ply(final_clusterscene, normalized_rgb):
    
    i = 0
    for i in range(cnn_prediction): # go trough all clusters

        final_segment = final_clusterscene[np.where(final_clusterscene[:,6] == i)]
        final_segment = numpy_to_pyntcloud(final_segment, normalized_rgb)
        
        path = "cluster_"+str(i)+".ply"
        final_segment.to_file(path)
        
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Cluster to mesh BETA. This function doesn't work yet. 
# Maybe because the convex hull algorithm of pyntcloud makes the mesh waterproof. 

def cluster_to_mesh(final_clusterscene):
    i = 0
    for i in range(cnn_prediction): # go trough all clusters

        final_segment = final_clusterscene[np.where(final_clusterscene[:,6] == i)]
        final_segment = numpy_to_pyntcloud(final_segment, normalized_rgb)
        
        # Create mesh Pyntcloud structure
        mesh_id= final_segment.add_structure("convex_hull", qhull_options="Qt")
        segment_mesh = final_segment.structures[mesh_id]

        segment_mesh = segment_mesh.get_mesh() # Creates a dataframe with v1,v2,v3

        final_mesh = PyntCloud(df_final_segment, mesh=segment_mesh)

        path = "cluster"+str(i)+".ply"
        final_mesh.to_file(path, also_save=["mesh"])

CNN Model with Voxelsize of 256 has been loaded.


In [2]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Start  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

DATADIR = "data\lidar\\"
file = "KleinerKonfiEcke_010.ply"
path = DATADIR + file

pointcloud = PyntCloud.from_file(path)  #Format: x,y,z, r, g, b
initial_pointcloud = PyntCloud.from_file(path) # Stays untouched for error calculation


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Input raw Pointcloud into CNN ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Voxelgrid only takes the XYZ Informations
voxelgrid_id = pointcloud.add_structure("voxelgrid", n_x = voxel_size, n_y = voxel_size, n_z = voxel_size)
voxelscene = pointcloud.structures[voxelgrid_id]

# Create binary array from Voxelscene
binary_voxelscene = voxelscene.get_feature_vector(mode="binary")

# Prepare data for Network input
# A 3D Cnn expects an 4D Tensor as Input 
binary_voxelscene = np.expand_dims(binary_voxelscene, axis=0)

# CNN prediction
cnn_out = model.predict(binary_voxelscene)
cnn_prediction = np.argmax(cnn_out)

print("Predicted Numbers of Objects:", cnn_prediction)

#voxelscene.plot(d=3, mode="density", cmap="hsv")

Predicted Numbers of Objects: 15


In [3]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Prepare Pointcloud for Clustering ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

n_points = pointcloud.points.shape[0]

# Downsamling for better performance. 
# 25000 is only for showcasing the differen cluster candidates. 
# For real usage 250.000 points with candidates = 1 is a good start
if(n_points > 2500):
    downsample_coeff = n_points / 2500
    
    print("Pointcloud with", n_points, "points is too big for segmentation.")
    print("Downsampling the Pointcloud to", round(n_points/downsample_coeff)," points." )
    
    pointcloud = downsample_pointcloud(pointcloud.points, round(downsample_coeff))
    pointcloud = pointcloud.to_numpy()
else: 
    pointcloud = pointcloud.points.to_numpy()


# Preparing Pointcloud for input in o3d. Colors have to be in r,g,b and in range 0-1    
p_x = pointcloud[:, 0]
p_y = pointcloud[:, 1]
p_z = pointcloud[:, 2]

p_r = pointcloud[:, 3] / 255
p_g = pointcloud[:, 4] / 255 
p_b = pointcloud[:, 5] / 255

df_pointcloud_xyz = pd.DataFrame(zip(p_x, p_y, p_z), columns=['x', 'y', 'z']) 
df_pointcloud_rgb = pd.DataFrame(zip(p_r, p_g, p_b), columns=['r','g','b']) 

pointcloud_xyz = df_pointcloud_xyz.to_numpy()
pointcloud_xyz = pc_normalize(pointcloud_xyz)
pointcloud_rgb = df_pointcloud_rgb.to_numpy()

# Create Outliner Removal o3 
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pointcloud_xyz)
pcd.colors = o3d.utility.Vector3dVector(pointcloud_rgb)

# the lower std_ration the more agressive is the outliner removal
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=80, std_ratio=3.0)

#display_inlier_outlier(pcd, ind)

#back to numpy array
pointcloud_xyz = np.asarray(cl.points)
pointcloud_rgb = np.asarray(cl.colors)

x = (pointcloud_xyz[:, 0])
y = (pointcloud_xyz[:, 1])
z = (pointcloud_xyz[:, 2])

r = (pointcloud_rgb[:, 0]) *255
g = (pointcloud_rgb[:, 1]) *255
b = (pointcloud_rgb[:, 2]) *255

# In Pyntcloud color has to be 'red', 'green', 'blue' and in range 0-255
df_pointcloud = pd.DataFrame(zip(x, y, z, r, g, b), columns=['x', 'y', 'z','red','green','blue'])
pointcloud = PyntCloud(df_pointcloud)

print("Pointcloud after downsampling and outliner removal")
pointcloud.plot(initial_point_size=0.014)    

Pointcloud with 917756 points is too big for segmentation.
Downsampling the Pointcloud to 2500  points.
Pointcloud after downsampling and outliner removal


C:\Users\Florian\.conda\envs\busch\lib\site-packages\pythreejs\traits.py:191: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.007297606143681331, 0.7899786032540111, 0…

In [4]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ CLUSTERING ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#Prepare Pointcloud for Clustering
cluster_pointcloud = pointcloud.xyz

# Setting up the Clustering Candidates
gaussian_clustering = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-5, max_iter=600)
bayesian_clustering = BayesianGaussianMixture(n_components=cnn_prediction, 
                                              covariance_type='full',tol=1e-5, max_iter=600)

gaussian_clustering2 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-6, max_iter=700)
bayesian_clustering2 = BayesianGaussianMixture(n_components=cnn_prediction, 
                                               covariance_type='full',tol=1e-6, max_iter=700)

gaussian_clustering3 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-6, max_iter=800)
bayesian_clustering3 = BayesianGaussianMixture(n_components=cnn_prediction, 
                                               covariance_type='full',tol=1e-6, max_iter=800)

gaussian_clustering4 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-7, max_iter=1000)
bayesian_clustering4= BayesianGaussianMixture(n_components=cnn_prediction, 
                                              covariance_type='full',tol=1e-7, max_iter=1000)

gaussian_clustering5 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-8, max_iter=1500)
bayesian_clustering5 = BayesianGaussianMixture(n_components=cnn_prediction, 
                                              covariance_type='full',tol=1e-8, max_iter=1500)

gaussian_clustering6 = GaussianMixture(n_components=cnn_prediction, covariance_type='full',tol=1e-9, max_iter=2000)
bayesian_clustering6 = BayesianGaussianMixture(n_components=cnn_prediction,
                                              covariance_type='full',tol=1e-9, max_iter=2000)

gaussian_clustering7 = GaussianMixture(n_components=cnn_prediction, covariance_type='diag',tol=1e-4, max_iter=500)
bayesian_clustering7 = BayesianGaussianMixture(n_components=cnn_prediction,
                                              covariance_type='diag',tol=1e-4, max_iter=500)

gaussian_clustering8 = GaussianMixture(n_components=cnn_prediction, covariance_type='diag',tol=1e-6, max_iter=800)
bayesian_clustering8 = BayesianGaussianMixture(n_components=cnn_prediction,
                                              covariance_type='diag',tol=1e-6, max_iter=800)

# List of Labels per Clustering
gm ={}
gm_labels={}
bm={}
bm_labels={}

gm_clusters_dict={}
gm_clusterscene_rgb={}
gm_clusterscene_labels={}

bm_clusters_dict={}
bm_clusterscene_rgb={}
bm_clusterscene_labels={}

gm_clusterscene_rgb_pynt={}
bm_clusterscene_rgb_pynt ={}


i = 0
j = 0

while i <= n_candidates:
    
    if i == 0:
        print("First candidate and second candidate:")
        gm[i] = gaussian_clustering.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering.predict(cluster_pointcloud) 
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 1:
        print("Thrid candidate and fourth candidate:")
        gm[i] = gaussian_clustering2.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering2.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering2.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering2.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 2:
        print("Fifth candidate and sixths candidate:")
        gm[i] = gaussian_clustering3.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering3.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering3.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering3.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 3:
        print("Seventh candidate and eights candidate:")
        gm[i] = gaussian_clustering4.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering4.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering4.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering4.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 4:
        print("Ninth candidate and tenth candidate:")        
        gm[i] = gaussian_clustering5.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering5.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering5.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering5.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    if i == 5:
        print("Eleventh candidate and twelfth candidate:")        
        gm[i] = gaussian_clustering6.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering6.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering6.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering6.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
    
    if i == 6:
        print("Thirteeth candidate and fourteenth candidate:")        
        gm[i] = gaussian_clustering7.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering7.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering7.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering7.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
    
    if i == 5:
        print("Fifteenth candidate and sixteenth candidate:")        
        gm[i] = gaussian_clustering8.fit(cluster_pointcloud)
        gm_labels[i] = gaussian_clustering8.predict(cluster_pointcloud)
        bm[i] = bayesian_clustering8.fit(cluster_pointcloud)
        bm_labels[i] = bayesian_clustering8.predict(cluster_pointcloud)
        gm_clusters_dict[i] = {j: cluster_pointcloud[np.where(gm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        bm_clusters_dict[i] = {j: cluster_pointcloud[np.where(bm_labels[i] == j)[0]] for j in range(cnn_prediction)}
        
    # Colorize Clusters and adding Clusterlabels
    
    gm_clusterscene_rgb[i], gm_clusterscene_labels[i] = colorize_to_dataframe(gm_clusters_dict[i], cnn_prediction)
    bm_clusterscene_rgb[i], bm_clusterscene_labels[i] = colorize_to_dataframe(bm_clusters_dict[i], cnn_prediction)
    
    gm_clusterscene_rgb_pynt[i] = PyntCloud(gm_clusterscene_rgb[i])
    bm_clusterscene_rgb_pynt[i] = PyntCloud(bm_clusterscene_rgb[i])
    
    gm_clusterscene_rgb_pynt[i].plot(initial_point_size=0.1)
    bm_clusterscene_rgb_pynt[i].plot(initial_point_size=0.1)
    
    print("Cluster", (i+1)*2, "from", (n_candidates+1)*2, "done.")

    i=i+1
    
print("Segmentation done")

First candidate and second candidate:


C:\Users\Florian\.conda\envs\busch\lib\site-packages\pythreejs\traits.py:191: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.007297606143681331, 0.7899786032540111, 0…

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.007297606143681342, 0.7899786032540111, 0…

Cluster 2 from 8 done.
Thrid candidate and fourth candidate:


...

Cluster 6 from 8 done.
Seventh candidate and eights candidate:


C:\Users\Florian\.conda\envs\busch\lib\site-packages\pythreejs\traits.py:191: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.007297606143681353, 0.7899786032540111, 0…

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.007297606143681336, 0.7899786032540111, 0…

Cluster 8 from 8 done.
Segmentation done


In [12]:
# Cluster Validation 
i = 0
max_calinski_score = 0
max2_calinski_score = 0 # Second best score

gm_calinski_score = []
bm_calinski_score = []

while i <= n_candidates:
    
    gm_calinski_score.append(calinski_harabasz_score(cluster_pointcloud, gm_clusterscene_labels[i][:,3]))
    bm_calinski_score.append(calinski_harabasz_score(cluster_pointcloud, bm_clusterscene_labels[i][:,3]))
    
    #print(gm_calinski_score[i], bm_calinski_score[i]) 
    i = i + 1
    
# get max   
gm_max =  max(gm_calinski_score)

gm_calinski_score_temp = set(gm_calinski_score) 
# removing the largest element from temp list 
gm_calinski_score_temp.remove(gm_max) 
# get 2nd max
gm_2max = max(gm_calinski_score_temp)

#-----------------------------------------------
# get max 
bm_max =  max(bm_calinski_score)

bm_calinski_score_temp = set(bm_calinski_score) 
bm_calinski_score_temp.remove(bm_max) 
# get 2nd max
bm_2max = max(bm_calinski_score_temp)

if (gm_max > bm_max):
    index_max = gm_calinski_score.index(gm_max)
    best_clusterscene = gm_clusterscene_labels[index_max]
else: 
    index_max = bm_calinski_score.index(bm_max)
    best_clusterscene = bm_clusterscene_labels[index_max]

if (gm_2max > bm_2max):
    index_2max = gm_calinski_score.index(gm_2max)
    best2_clusterscene = gm_clusterscene_labels[index_2max]
else: 
    index_2max = bm_calinski_score.index(bm_2max)
    best2_clusterscene = bm_clusterscene_labels[index_2max]

#Check
print("Best Score:", calinski_harabasz_score(cluster_pointcloud, best_clusterscene[:,3]))
print("2nd Best Score:", calinski_harabasz_score(cluster_pointcloud, best2_clusterscene[:,3]))

Best Score: 247.2061640783901
2nd Best Score: 215.90003819293668


In [6]:
# Get Assignments and Error Calc for best Cluster
gm_best_assignments = get_assignments(best_clusterscene, gm_clusterscene_labels)
bm_best_assignments = get_assignments(best_clusterscene, bm_clusterscene_labels)

# Get Assignments and Error Calc for 2nd best Cluster
gm_best2_assignments = get_assignments(best2_clusterscene, gm_clusterscene_labels)
bm_best2_assignments = get_assignments(best2_clusterscene, bm_clusterscene_labels)

In [11]:
gm_best_assignments

defaultdict(dict,
            {0: {0: ([81], [132], [0], [8], [61]),
              1: ([49], [83], [1], [5], [59]),
              2: ([385], [385], [2], [12], [100]),
              3: ([81], [81], [3], [13], [100]),
              4: ([112], [113], [4], [0], [99]),
              5: ([267], [357], [5], [11], [75]),
              6: ([243], [245], [6], [13], [99]),
              7: ([38], [39], [7], [3], [97]),
              8: ([45], [45], [8], [4], [100]),
              9: ([159], [162], [9], [7], [98]),
              10: ([85], [101], [10], [9], [84]),
              11: ([309], [385], [11], [14], [80]),
              12: ([51], [95], [12], [0], [54]),
              13: ([33], [34], [13], [2], [97]),
              14: ([200], [200], [14], [6], [100])},
             1: {0: ([56], [132], [0], [2], [42]),
              1: ([83], [83], [1], [3], [100]),
              2: ([384], [385], [2], [14], [100]),
              3: ([81], [81], [3], [10], [100]),
              4: ([101], [113], [4], [1

In [7]:
homgenity_check_best = homogenity_check(gm_best_assignments, bm_best_assignments)
homgenity_check_best2 = homogenity_check(gm_best2_assignments, bm_best2_assignments)

print("Homogenity best Cluster:", homgenity_check_best,"% to the other Clusterings")
print("Homogenity 2nd best Cluster:", homgenity_check_best2,"% to the other Clusterings")

Homogenity best Cluster: 84.61 % to the other Clusterings
Homogenity 2nd best Cluster: 85.36 % to the other Clusterings


In [8]:
if(homgenity_check_best >= homgenity_check_best2):
    print("Continuing with best Cluster.")
    final_clusterscene = restore_real_rgb(best_clusterscene, pointcloud)
    
elif(homgenity_check_best2 > homgenity_check_best):
    print("Continuing with 2nd best Cluster.")
    final_clusterscene = restore_real_rgb(best2_clusterscene, pointcloud)

Continuing with 2nd best Cluster.


In [9]:
# Export clusters to file
export_to_ply(final_clusterscene, normalized_rgb=1)

# Export full scene to file
full_final_clusterscene = numpy_to_pyntcloud(final_clusterscene, normalized_rgb=1)
path = "full_clusterscene_labeled.ply"
full_final_clusterscene.to_file(path)

print("Automatic segmentation of pointcloud finished.")

Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Creating an X,Y,Z,R,G,B,Label Pyntcloud. With normalized RGB.
Automati

In [10]:
# Plot selected Segment. Check if color assignment went right

#final_segment = final_clusterscene[np.where(final_clusterscene[:,6] == 0)]
#final_segment = numpy_to_pyntcloud(final_segment)
#final_segment.plot()

#cluster_to_mesh(final_clusterscene)